In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cykVBLOaxqUNzCreAcz2uhf6XkywmQKgr1NvuAnozBJW',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'loanamountprediction-donotdelete-pr-2emz3xvikbsonf'
object_key = 'loan1.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset= pd.read_csv(body)
dataset.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,Coapplicant_Income,LoanAmount,Loan,Loan_Amount_Term,Credit_History,Emi,Loan_Status,Alloted_amount
0,LP001002,Male,No,0.0,Graduate,No,52690,0.0,100000,100000.0,12,1.0,920.594223,Y,79035.0
1,LP001003,Male,Yes,1.0,Graduate,No,45830,15080.0,18000,18000.0,18,1.0,202.227276,N,91365.0
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,85000,85000.0,18,1.0,954.962133,Y,10800.0
3,LP001006,Male,Yes,0.0,Not Graduate,No,25830,23580.0,100000,100000.0,18,1.0,1123.242469,Y,74115.0
4,LP001008,Male,No,0.0,Graduate,No,60000,0.0,150000,150000.0,18,1.0,1685.227303,Y,90000.0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

dataset.isnull()
d1=dataset.fillna(method='bfill')
print(d1['Credit_History'])

x=d1[['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','Coapplicant_Income','LoanAmount','Loan_Amount_Term','Credit_History','Emi']]
y=d1['Alloted_amount']
print(x)

#gender=pd.get_dummies(x['Gender'])
#x=x.drop('Gender',axis=1)
#x=pd.concat([x,gender],axis=1)
#print(x)
df_dummies = pd.get_dummies(x, prefix='', prefix_sep='', columns=['Gender', 'Married','Education','Self_Employed'])
#df_dummies.head()
x=x.drop(['Dependents', 'ApplicantIncome', 'Coapplicant_Income', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Emi','Gender','Married','Education','Self_Employed'],axis=1)
x=pd.concat([x,df_dummies],axis=1)
print(list(x.columns))
fea_list=list(x.columns)
print(fea_list)


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=50)

from sklearn.tree import DecisionTreeRegressor 
# Instantiate model with 1000 decision trees
rf = DecisionTreeRegressor()
# Train the model on training data
rf.fit(x_train,y_train);

predictions=rf.predict(x_test)
# Calculate the absolute errors
errors = abs(predictions -y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

print(rf.predict([[0,50000,42000,200000,60,0,3000,0,1,0,1,0,1,1,0]]))
#print("root_mean_sqrd_error is=",np.sqrt(mean_squared_error(y_test,predictions)))

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
609    1.0
610    1.0
611    1.0
612    1.0
613    0.0
Name: Credit_History, Length: 614, dtype: float64
     Gender Married  Dependents     Education Self_Employed  ApplicantIncome  \
0      Male      No         0.0      Graduate            No            52690   
1      Male     Yes         1.0      Graduate            No            45830   
2      Male     Yes         0.0      Graduate           Yes             3000   
3      Male     Yes         0.0  Not Graduate            No            25830   
4      Male      No         0.0      Graduate            No            60000   
..      ...     ...         ...           ...           ...              ...   
609  Female      No         0.0      Graduate            No            29000   
610    Male     Yes         3.0      Graduate            No            41060   
611    Male     Yes         1.0      Graduate            No            80720   
612    Male     Yes         2

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


# Deployment

In [10]:
pip install -U ibm_watson_machine_learning

Note: you may need to restart the kernel to use updated packages.


In [9]:
from ibm_watson_machine_learning import APIClient
import json

In [11]:
cred={"apikey":"UFiKYlV2AoWKo8kRnD98FYnw1kbFf1pjM9tjqIRhQplY","url":"https://us-south.ml.cloud.ibm.com"}

In [12]:
client=APIClient(cred)
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------------  ------------------------
ID                                    NAME             CREATED
9575debf-c085-4882-b3e0-ce792ea0ddf0  Loan prediction  2022-11-16T06:38:35.657Z
------------------------------------  ---------------  ------------------------


In [14]:
space_id="9575debf-c085-4882-b3e0-ce792ea0ddf0"
client.set.default_space(space_id)

'SUCCESS'

In [15]:
client.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

# Save and Deploy

In [17]:
import sklearn
sklearn.__version__

'1.0.2'

In [18]:
MODEL_NAME="loan prediction"
DEPLOYMENT_NAME="Loan prediction"
DEMO_MODEL=rf

In [19]:
ssid=client.software_specifications.get_id_by_name("runtime-22.1-py3.9")

In [21]:
model_props={
    client.repository.ModelMetaNames.NAME:MODEL_NAME,
    client.repository.ModelMetaNames.TYPE:'scikit-learn_1.0',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:ssid

}

In [22]:
model_details=client.repository.store_model(
model=DEMO_MODEL,
meta_props=model_props,
training_data=x_train,
training_target=y_train
)

In [23]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'Alloted_amount',
  'schemas': {'input': [{'fields': [{'name': 'Dependents', 'type': 'float64'},
      {'name': 'ApplicantIncome', 'type': 'int64'},
      {'name': 'Coapplicant_Income', 'type': 'float64'},
      {'name': 'LoanAmount', 'type': 'int64'},
      {'name': 'Loan_Amount_Term', 'type': 'int64'},
      {'name': 'Credit_History', 'type': 'float64'},
      {'name': 'Emi', 'type': 'float64'},
      {'name': 'Female', 'type': 'uint8'},
      {'name': 'Male', 'type': 'uint8'},
      {'name': 'No', 'type': 'uint8'},
      {'name': 'Yes', 'type': 'uint8'},
      {'name': 'Graduate', 'type': 'uint8'},
      {'name': 'Not Graduate', 'type': 'uint8'},
      {'name': 'No', 'type': 'uint8'},
      {'name': 'Yes', 'type': 'uint8'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'me

In [24]:
model_id=client.repository.get_model_id(model_details)
model_id

'8b8971dc-b1e9-498f-aaa4-e9f99e6f59b8'

In [25]:
deployment_props={
    client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME,
    client.deployments.ConfigurationMetaNames.ONLINE:{}
}

In [26]:
deployment=client.deployments.create(
artifact_uid=model_id,
meta_props=deployment_props
)



#######################################################################################

Synchronous deployment creation for uid: '8b8971dc-b1e9-498f-aaa4-e9f99e6f59b8' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='53c0255e-5264-4c30-8c99-7d4f79629557'
------------------------------------------------------------------------------------------------


